In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip "drive/My Drive/Proyecto IA II/kaggle_bee_vs_wasp.zip"

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: kaggle_bee_vs_wasp/other_noinsect/576855.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576993.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577087.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577126.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577128.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577283.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577310.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577643.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577807.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577847.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578261.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578295.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578441.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578580.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578650.jpg  
  inflating: kaggle_bee_vs_wasp/o

In [ ]:
#@title **Librerias**
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report, roc_auc_score

In [ ]:
#@title **Información del dataset**
metadata = pd.read_csv("/content/kaggle_bee_vs_wasp/labels.csv")
metadata.head()

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,bee
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,bee
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,bee
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,bee
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,bee


In [ ]:
#@title **Separar datset en 80%, 10% y 10%**

train, val_test = train_test_split(metadata, test_size=0.2, stratify = metadata["label"])
validation, test = train_test_split(val_test, test_size=0.5, stratify = val_test["label"])

metadata_labels = metadata["label"]
metadata_labels_bee = metadata_labels[metadata_labels=="bee"]
metadata_labels_insect = metadata_labels[metadata_labels=="insect"]
metadata_labels_other = metadata_labels[metadata_labels=="other"]
metadata_labels_wasp = metadata_labels[metadata_labels=="wasp"]

print("Total bees: ", str(metadata_labels_bee.count()))
print("Total insects: ", str(metadata_labels_insect.count()))
print("Total others: ", str(metadata_labels_other.count()))
print("Total wasps: ", str(metadata_labels_wasp.count()))
print("")
print("Train: " + str(train.shape[0]))
print("Validation: " + str(validation.shape[0]))
print("Test: " + str(test.shape[0]))
print("")
print("Train: " + str(train.shape[0] / metadata.shape[0]))
print("Validation: " + str(validation.shape[0] / metadata.shape[0]))
print("Test: " + str(test.shape[0] / metadata.shape[0]))

Total bees:  3183
Total insects:  2439
Total others:  856
Total wasps:  4943

Train: 9136
Validation: 1142
Test: 1143

Train: 0.7999299535942562
Validation: 0.09999124419928203
Test: 0.10007880220646179


In [ ]:
#@title **Crear las carpetas para la distribución**

base_dir = "base_dir"
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, "image_train")
os.mkdir(train_dir)

val_dir = os.path.join(base_dir, "image_val")
os.mkdir(val_dir)

test_dir = os.path.join(base_dir, "image_test")
os.mkdir(test_dir)

labels = list(metadata["label"].unique())

for label in labels:
    label_path_train = os.path.join(train_dir, label)
    os.mkdir(label_path_train)
    label_path_val = os.path.join(val_dir, label)
    os.mkdir(label_path_val)
    label_path_test = os.path.join(test_dir, label)
    os.mkdir(label_path_test)

In [ ]:
#@title **Copiar las imagenes al nuevo directorio**

image_dir = "/content/kaggle_bee_vs_wasp"

for i in train["id"].values:
    image_folder, image_name = train["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(train_dir, train["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)

for i in validation["id"].values:
    image_folder, image_name = validation["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(val_dir, validation["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)
    
for i in test["id"].values:
    image_folder, image_name = test["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(test_dir, test["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)


train_bee = np.shape(os.listdir("/content/base_dir/image_train/bee"))[0]
train_insect = np.shape(os.listdir("/content/base_dir/image_train/insect"))[0]
train_wasp = np.shape(os.listdir("/content/base_dir/image_train/wasp"))[0]
train_other = np.shape(os.listdir("/content/base_dir/image_train/other"))[0]
train_total = train_bee + train_insect + train_wasp + train_other

validation_bee = np.shape(os.listdir("/content/base_dir/image_val/bee"))[0]
validation_insect = np.shape(os.listdir("/content/base_dir/image_val/insect"))[0]
validation_wasp = np.shape(os.listdir("/content/base_dir/image_val/wasp"))[0]
validation_other = np.shape(os.listdir("/content/base_dir/image_val/other"))[0]
validation_total = validation_bee + validation_insect + validation_wasp + validation_other

test_bee = np.shape(os.listdir("/content/base_dir/image_test/bee"))[0]
test_insect = np.shape(os.listdir("/content/base_dir/image_test/insect"))[0]
test_wasp = np.shape(os.listdir("/content/base_dir/image_test/wasp"))[0]
test_other = np.shape(os.listdir("/content/base_dir/image_test/other"))[0]
test_total = test_bee + test_insect + test_wasp + test_other

print("Total train bee images: " + str(train_bee))
print("Total train insect images: " + str(train_insect))
print("Total train wasp images: " + str(train_wasp))
print("Total train other images: " + str(train_other))
print("")
print("Total validation bee images: " + str(validation_bee))
print("Total validation insect images: " + str(validation_insect))
print("Total validation wasp images: " + str(validation_wasp))
print("Total validation other images: " + str(validation_other))
print("")
print("Total test bee images: " + str(test_bee))
print("Total test insect images: " + str(test_insect))
print("Total test wasp images: " + str(test_wasp))
print("Total test other images: " + str(test_other))
print("")
print("Total train images: " + str(train_total))
print("Total validation images: " + str(validation_total))
print("Total test images: " + str(test_total))

Total train bee images: 2546
Total train insect images: 1951
Total train wasp images: 3954
Total train other images: 685

Total validation bee images: 318
Total validation insect images: 244
Total validation wasp images: 494
Total validation other images: 86

Total test bee images: 319
Total test insect images: 244
Total test wasp images: 495
Total test other images: 85

Total train images: 9136
Total validation images: 1142
Total test images: 1143


In [ ]:
#@title **Data Augmentation**

data_gen_param = {
    #"rotation_range": 180,
    "width_shift_range": 0.1,
    "height_shift_range": 0.1,
    "zoom_range": 0.1,
    "horizontal_flip": True
    #"vertical_flip": True
}
data_generator = ImageDataGenerator(**data_gen_param)
num_images_each_label = 4000

aug_dir = os.path.join(base_dir, "aug_dir")
os.mkdir(aug_dir)
num_images = 0

for label in labels:
    
    img_dir = os.path.join(aug_dir, "aug_img")
    os.mkdir(img_dir)
    
    src_dir_label = os.path.join(train_dir, label)
    for image_name in os.listdir(src_dir_label):
        shutil.copy(os.path.join(src_dir_label, image_name), os.path.join(img_dir, image_name))
    
    batch_size = 32
    data_flow_param = {
        "directory": aug_dir,
        "color_mode": "rgb",
        "batch_size": batch_size,
        "shuffle": True,
        "save_to_dir": os.path.join(train_dir, label),
        "save_format": "jpg"
    }
    aug_data_gen = data_generator.flow_from_directory(**data_flow_param)
    
    num_img_aug = num_images_each_label - len(os.listdir(os.path.join(train_dir, label)))
    num_batch = int(num_img_aug / batch_size)
    
    for i in range(0, num_batch):
        next(aug_data_gen)
    
    shutil.rmtree(img_dir)
    num_images = num_images + 1

#shutil.rmtree(aug_dir)

#Now check if the data is balanced.

for label in labels:
    print(label + " train: " + str(len(os.listdir(os.path.join(train_dir, label)))))
print("\n")
for label in labels:
    print(label + " val: " + str(len(os.listdir(os.path.join(val_dir, label)))))

Found 2546 images belonging to 1 classes.
Found 3954 images belonging to 1 classes.
Found 1951 images belonging to 1 classes.
Found 685 images belonging to 1 classes.
bee train: 3986
wasp train: 3986
insect train: 3998
other train: 3905


bee val: 318
wasp val: 494
insect val: 244
other val: 86


In [ ]:
#Comprimir dataset

!zip -r data_aug_4_classes.zip base_dir

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: base_dir/image_train/insect/42384942895_654682ffc2_n.jpg (deflated 3%)
  adding: base_dir/image_train/insect/_655_5530647.jpg (deflated 2%)
  adding: base_dir/image_train/insect/_1240_4776834.jpg (deflated 1%)
  adding: base_dir/image_train/insect/_1094_3530858.jpg (deflated 2%)
  adding: base_dir/image_train/insect/_286_8359104.jpg (deflated 1%)
  adding: base_dir/image_train/insect/27466882187_1233cb9f79_m.jpg (deflated 0%)
  adding: base_dir/image_train/insect/_1115_8115449.jpg (deflated 2%)
  adding: base_dir/image_train/insect/3791132058_00669c7924_n.jpg (deflated 0%)
  adding: base_dir/image_train/insect/8709079109_850c548a49_n.jpg (deflated 3%)
  adding: base_dir/image_train/insect/34616636634_3d10c5eb5b_n.jpg (deflated 0%)
  adding: base_dir/image_train/insect/2798040684_7425ed042c_m.jpg (deflated 0%)
  adding: base_dir/image_train/insect/_1129_319270.jpg (deflated 1%)
  adding: base_dir/image_train/insect/_

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [ ]:
#Autenticar en el drive para copiar

auth.authenticate_user()

In [ ]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

  media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created

In [ ]:
#Save file in Drive

save_file_to_drive('data_aug_4_classes.zip', 'data_aug_4_classes.zip')

File ID: 1f4HK7BjIQw4GeLZ-uKoYZwZJG-gKEwUF


{'id': '1f4HK7BjIQw4GeLZ-uKoYZwZJG-gKEwUF'}